# Rule-Based Graph Representations of Cellular Automata

In [73]:
from itertools import product

# ---- 1) GLOBAL PARAMETERS (edit these) ---------------------
RULE   = 30   # Wolfram code in [0..255] for radius-1 binary CA
RADIUS = 1    # Neighborhood radius r; r=1 -> 3-neighborhood

# ---- 2) LOCAL MAPPING CONSTRUCTION -------------------------
def build_rule_map(rule_number: int = RULE, r: int = RADIUS):
    """
    Return the local neighborhood-to-output lookup for a 1D binary CA.

    For radius r, the local neighborhood has length L = 2r + 1.
    For r=1 (the standard “elementary” case), L=3 and there are 2^3=8
    neighborhoods in total, from '000' to '111' (lexicographic order).

    Wolfram encoding reminder (elementary CA):
      - The 8 output bits of 'rule_number' correspond to neighborhoods
        read from 111 down to 000 (in that order).
      - To index neighborhoods in ascending lexicographic order
        (000,001,...,111), we reverse the bitstring (rule_bits[::-1]),
        so `rule_bits[i]` matches `neighborhoods[i]`.

    Returns
    -------
    dict: { neighborhood (str of length 2r+1) : output_bit (str '0' or '1') }
    """
    L = 2 * r + 1                     # neighborhood length
    table_size = 2 ** L               # number of neighborhoods
    # Binary string of the rule, padded to table_size bits.
    # Example (r=1): for RULE=30, bin is '00011110' (111..000 order).
    # We reverse it so index i matches neighborhoods in lexicographic order.
    rule_bits = f"{rule_number:0{table_size}b}"[::-1]

    # Neighborhoods in lexicographic order: '000', '001', ..., '111'
    neighborhoods = [''.join(p) for p in product('01', repeat=L)]

    # Map each neighborhood to its output bit
    return {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

# ---- 3) BUILD AND REPORT THE RULE TABLE --------------------
RULE_MAP = build_rule_map(RULE, RADIUS)

# ---- 4) VERBOSE, HUMAN-FRIENDLY OUTPUT ---------------------
alphabet = "{0,1}"
L = 2 * RADIUS + 1
table_size = 2 ** L
wolfram_bits = f"{RULE:0{table_size}b}"          # 111..000 order
lexicographic_bits = wolfram_bits[::-1]          # 000..111 order

print("============================================================")
print(" Cellular Automaton Local Rule (Global Settings)")
print("------------------------------------------------------------")
print(f" Rule (Wolfram code) : {RULE}")
print(f" Alphabet            : {alphabet}")
print(f" Radius r            : {RADIUS}")
print(f" Neighborhood length : L = 2r + 1 = {L}")
print(f" Neighborhoods      : 2^{L} = {table_size}")
print()
print(" Wolfram bitstring (indexing neighborhoods from 111 down to 000):")
print(f"   {wolfram_bits}")
print(" Same bits aligned with lexicographic neighborhoods (000..111):")
print(f"   {lexicographic_bits}")
print(" Note: RULE_MAP indexes neighborhoods in lexicographic order.")
print("       That is, RULE_MAP['000'] corresponds to the leftmost bit")
print("       of the reversed bitstring shown just above.")
print("============================================================")
print(" Local transition table (neighborhood -> next-state output):")
for nb in sorted(RULE_MAP.keys()):  # '000' .. '111'
    print(f"   {nb} -> {RULE_MAP[nb]}")
print("============================================================")

 Cellular Automaton Local Rule (Global Settings)
------------------------------------------------------------
 Rule (Wolfram code) : 30
 Alphabet            : {0,1}
 Radius r            : 1
 Neighborhood length : L = 2r + 1 = 3
 Neighborhoods      : 2^3 = 8

 Wolfram bitstring (indexing neighborhoods from 111 down to 000):
   00011110
 Same bits aligned with lexicographic neighborhoods (000..111):
   01111000
 Note: RULE_MAP indexes neighborhoods in lexicographic order.
       That is, RULE_MAP['000'] corresponds to the leftmost bit
       of the reversed bitstring shown just above.
 Local transition table (neighborhood -> next-state output):
   000 -> 0
   001 -> 1
   010 -> 1
   011 -> 1
   100 -> 1
   101 -> 0
   110 -> 0
   111 -> 0


#  I. Rule-Space Representations（规则空间图）

## **1. De Bruijn Graph (dBG): A Rule-Space Blueprint for Local Syntax**
The De Bruijn Graph (dBG) encodes a cellular automaton’s local transition function as a finite-state graph structure that captures how neighborhood patterns overlap and propagate. Each vertex represents a length-$2r$ context string, and a directed edge connects two vertices if they share $2r{-}1$ symbols in common; the edge is labeled with the rule’s output for that $2r{+}1$-cell neighborhood. This compact automaton translates the local rule into a formal grammar of admissible configurations, providing an algorithmic representation of the CA’s intrinsic syntax. By analyzing the graph’s connectivity and cycles, researchers can deduce global properties such as surjectivity (the absence of “Garden of Eden” states) and reversibility (unique preimages). The dBG thus serves as a static “rule-space blueprint,” linking cellular automata to symbolic dynamics, information theory, and automata-based computational reasoning.

In [50]:
from itertools import product

def build_dbg_data(rule_number=RULE, r=RADIUS):
    """
    Build the De Bruijn Graph (dBG) for the current global CA rule.

    Returns
    -------
    nodes : list[str]
        All 2r-bit binary strings, e.g. ['00','01','10','11'] for r=1.
    edges : list[dict]
        Each dict = {'from': str, 'to': str, 'label': int, 'neighborhood': str}.
    """
    L = 2*r + 1
    rule_bits = f"{rule_number:0{2**L}b}"[::-1]  # reverse for 000..111 order
    neighborhoods = [''.join(p) for p in product('01', repeat=L)]
    rule_map = {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

    nodes = [''.join(p) for p in product('01', repeat=2*r)]
    edges = []
    for nb, out in rule_map.items():
        src, dst = nb[:-1], nb[1:]
        edges.append({
            'from': src,
            'to': dst,
            'label': int(out),
            'neighborhood': nb
        })
    return nodes, edges


# ---- Build and print --------------------------------------------------------
nodes, edges = build_dbg_data()

print("============================================================")
print(f"De Bruijn Graph — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
print("NODES:")
for n in nodes:
    print(f"  {n}")
print(f"Total nodes: {len(nodes)}")

print("\nEDGES:")
for e in edges:
    print(f"  {e['from']} -> {e['to']}  [label={e['label']}]  (neighbor={e['neighborhood']})")
print(f"Total edges: {len(edges)}")
print("============================================================")

De Bruijn Graph — Rule 30 (radius=1)
------------------------------------------------------------
NODES:
  00
  01
  10
  11
Total nodes: 4

EDGES:
  00 -> 00  [label=0]  (neighbor=000)
  00 -> 01  [label=1]  (neighbor=001)
  01 -> 10  [label=1]  (neighbor=010)
  01 -> 11  [label=1]  (neighbor=011)
  10 -> 00  [label=1]  (neighbor=100)
  10 -> 01  [label=0]  (neighbor=101)
  11 -> 10  [label=0]  (neighbor=110)
  11 -> 11  [label=0]  (neighbor=111)
Total edges: 8


## **2. Pair / Subset / Cycle Graphs: Derived Analytical Constructions from the dBG**
The **Pair, Subset, and Cycle Graphs** are higher-order analytical extensions of the De Bruijn Graph, designed to transform its local syntactic structure into tools for formal reasoning about a cellular automaton’s **global properties**. Each construction operates on the same underlying neighborhood-overlap principle but repurposes it for specific proof-oriented objectives.

The **Pair Graph** synchronizes two parallel walks on the dBG—its nodes are ordered pairs of vertices $(v,w)$—and edges connect $(v,w)\!\to\!(x,y)$ whenever both original edges emit the **same output symbol**. This “coupled traversal” detects whether distinct preimage paths can produce identical outputs, thus offering a concrete method to test **injectivity and reversibility**.

The **Subset Graph** generalizes the NFA→DFA subset construction: nodes are **sets of dBG vertices**, and transitions map each set to the collection of possible successors under a fixed output symbol. By tracking which subsets become unreachable or terminal, researchers can formally verify **surjectivity** and identify “Garden of Eden” configurations—global states with no valid preimage.

The **Cycle Graph**, meanwhile, focuses internally on **recurrent loops** within the dBG or its derivatives. These cycles correspond to **periodic neighborhood sequences**—the local periodic words that underpin spatial or temporal regularities in the automaton’s evolution.

Together, these derived graphs extend the analytical power of the dBG from mere rule encoding to **logical inference over reversibility, surjectivity, and periodic structure**, forming a rigorous automata-theoretic framework for reasoning about global dynamics from purely local syntax.

In [51]:
from itertools import chain, combinations
import networkx as nx

# ------------------------------------------------------------
# 1) Pair Graph (for reversibility analysis)
# ------------------------------------------------------------
def build_pair_graph(rule_number=RULE, r=RADIUS):
    """
    Pair Graph synchronizes two dBG paths that emit the same output label.
    Node: ordered pair (v1,v2) of dBG nodes.
    Edge: (v1,v2) -> (x1,x2) if edges v1->x1 and v2->x2 share same label.
    """
    _, dbg_edges = build_dbg_data(rule_number, r)
    # Collect edges grouped by label for efficient pairing
    edges_by_label = {'0': [], '1': []}
    for e in dbg_edges:
        edges_by_label[str(e['label'])].append((e['from'], e['to']))

    # Generate all synchronized edges
    nodes = set()
    edges = []
    for label, pairs in edges_by_label.items():
        for (u1, v1) in pairs:
            for (u2, v2) in pairs:
                nodes.add((u1, u2))
                nodes.add((v1, v2))
                edges.append({
                    'from': (u1, u2),
                    'to': (v1, v2),
                    'label': int(label)
                })
    return sorted(list(nodes)), edges

pair_nodes, pair_edges = build_pair_graph()
print("============================================================")
print(f"PAIR GRAPH — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
print("Nodes (v1,v2):")
for n in pair_nodes:
    print(f"  {n}")
print(f"Total nodes: {len(pair_nodes)}")

print("\nEdges ((v1,v2)->(x1,x2)):")
for e in pair_edges:
    print(f"  {e['from']} -> {e['to']}  [label={e['label']}]")
print(f"Total edges: {len(pair_edges)}")

PAIR GRAPH — Rule 30 (radius=1)
------------------------------------------------------------
Nodes (v1,v2):
  ('00', '00')
  ('00', '01')
  ('00', '10')
  ('00', '11')
  ('01', '00')
  ('01', '01')
  ('01', '10')
  ('01', '11')
  ('10', '00')
  ('10', '01')
  ('10', '10')
  ('10', '11')
  ('11', '00')
  ('11', '01')
  ('11', '10')
  ('11', '11')
Total nodes: 16

Edges ((v1,v2)->(x1,x2)):
  ('00', '00') -> ('00', '00')  [label=0]
  ('00', '10') -> ('00', '01')  [label=0]
  ('00', '11') -> ('00', '10')  [label=0]
  ('00', '11') -> ('00', '11')  [label=0]
  ('10', '00') -> ('01', '00')  [label=0]
  ('10', '10') -> ('01', '01')  [label=0]
  ('10', '11') -> ('01', '10')  [label=0]
  ('10', '11') -> ('01', '11')  [label=0]
  ('11', '00') -> ('10', '00')  [label=0]
  ('11', '10') -> ('10', '01')  [label=0]
  ('11', '11') -> ('10', '10')  [label=0]
  ('11', '11') -> ('10', '11')  [label=0]
  ('11', '00') -> ('11', '00')  [label=0]
  ('11', '10') -> ('11', '01')  [label=0]
  ('11', '11') -> ('1

In [52]:
# ------------------------------------------------------------
# 2) Subset Graph (for surjectivity analysis)
# ------------------------------------------------------------
def powerset(iterable):
    """Generate all non-empty subsets of iterable."""
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))

def build_subset_graph(rule_number=RULE, r=RADIUS):
    """
    Subset Graph: DFA construction from dBG (NFA→DFA).
    Node: a subset of dBG nodes (as tuple of binary strings).
    Edge: subset S -> S' under output a ∈ {0,1},
          where S' = {v | ∃ u∈S, (u→v,label=a)}.
    """
    nodes_dbg, edges_dbg = build_dbg_data(rule_number, r)
    alphabet = ['0', '1']
    nodes = []
    edges = []

    for S in powerset(nodes_dbg):
        S = tuple(sorted(S))
        nodes.append(S)
        for a in alphabet:
            next_set = sorted({e['to'] for e in edges_dbg if e['from'] in S and str(e['label']) == a})
            if next_set:
                edges.append({
                    'from': S,
                    'to': tuple(next_set),
                    'label': int(a)
                })
    return nodes, edges

subset_nodes, subset_edges = build_subset_graph()
print("\n============================================================")
print(f"SUBSET GRAPH — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
print("Nodes (subsets of dBG nodes):")
for n in subset_nodes:
    print(f"  {n}")
print(f"Total nodes: {len(subset_nodes)}")

print("\nEdges (subset S -> subset S'):")
for e in subset_edges:
    print(f"  {e['from']} -> {e['to']}  [label={e['label']}]")
print(f"Total edges: {len(subset_edges)}")


SUBSET GRAPH — Rule 30 (radius=1)
------------------------------------------------------------
Nodes (subsets of dBG nodes):
  ('00',)
  ('01',)
  ('10',)
  ('11',)
  ('00', '01')
  ('00', '10')
  ('00', '11')
  ('01', '10')
  ('01', '11')
  ('10', '11')
  ('00', '01', '10')
  ('00', '01', '11')
  ('00', '10', '11')
  ('01', '10', '11')
  ('00', '01', '10', '11')
Total nodes: 15

Edges (subset S -> subset S'):
  ('00',) -> ('00',)  [label=0]
  ('00',) -> ('01',)  [label=1]
  ('01',) -> ('10', '11')  [label=1]
  ('10',) -> ('01',)  [label=0]
  ('10',) -> ('00',)  [label=1]
  ('11',) -> ('10', '11')  [label=0]
  ('00', '01') -> ('00',)  [label=0]
  ('00', '01') -> ('01', '10', '11')  [label=1]
  ('00', '10') -> ('00', '01')  [label=0]
  ('00', '10') -> ('00', '01')  [label=1]
  ('00', '11') -> ('00', '10', '11')  [label=0]
  ('00', '11') -> ('01',)  [label=1]
  ('01', '10') -> ('01',)  [label=0]
  ('01', '10') -> ('00', '10', '11')  [label=1]
  ('01', '11') -> ('10', '11')  [label=0]
  

In [53]:
# ------------------------------------------------------------
# 3) Cycle Graph (for periodic pattern analysis)
# ------------------------------------------------------------
def build_cycle_graph(rule_number=RULE, r=RADIUS):
    """
    Cycle Graph simply extracts the cycles (loops) in the dBG.
    Node: same as dBG node.
    Edge: same as dBG edge.
    Additional info: detected simple cycles (list of node sequences).
    """
    nodes, edges = build_dbg_data(rule_number, r)
    G = nx.DiGraph()
    for e in edges:
        G.add_edge(e['from'], e['to'], label=e['label'])
    cycles = list(nx.simple_cycles(G))
    return nodes, edges, cycles

cycle_nodes, cycle_edges, cycles = build_cycle_graph()
print("\n============================================================")
print(f"CYCLE GRAPH — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
print("Nodes:")
for n in cycle_nodes:
    print(f"  {n}")
print(f"Total nodes: {len(cycle_nodes)}")

print("\nEdges:")
for e in cycle_edges:
    print(f"  {e['from']} -> {e['to']}  [label={e['label']}]")
print(f"Total edges: {len(cycle_edges)}")

print("\nDetected cycles (closed walks):")
for c in cycles:
    print(f"  {' -> '.join(c)}")
print(f"Total cycles: {len(cycles)}")


CYCLE GRAPH — Rule 30 (radius=1)
------------------------------------------------------------
Nodes:
  00
  01
  10
  11
Total nodes: 4

Edges:
  00 -> 00  [label=0]
  00 -> 01  [label=1]
  01 -> 10  [label=1]
  01 -> 11  [label=1]
  10 -> 00  [label=1]
  10 -> 01  [label=0]
  11 -> 10  [label=0]
  11 -> 11  [label=0]
Total edges: 8

Detected cycles (closed walks):
  00
  11
  11 -> 10 -> 00 -> 01
  11 -> 10 -> 01
  10 -> 00 -> 01
  10 -> 01
Total cycles: 6


# II. State-Space Representations（状态空间图）

## **3. State-Transition Graph (STG): The Global Map of Cellular Automaton Dynamics**
The **State-Transition Graph (STG)** provides a complete, global representation of a cellular automaton’s evolution by treating every possible **configuration** of the system as a node and linking each to its successor under the global update rule $\Phi$. Specifically, for a system of $N$ cells with $k$ possible states per cell, the STG contains $k^N$ nodes, each representing a full spatial configuration $C_t$; a directed edge $C_t \!\to\! C_{t+1}$ is drawn whenever $\Phi(C_t) = C_{t+1}$.

The structure of the STG naturally decomposes into **basins of attraction**: each connected component funnels transient states into one or more **attractor cycles**—fixed points, periodic loops, or complex recurrent orbits. These basins collectively form a **landscape of global dynamics**, mapping how initial conditions determine long-term outcomes. Simple rules yield small, well-ordered basins, while complex or chaotic rules produce vast, tangled networks of transients and attractors.

Although its size grows exponentially with system scale, the STG remains the **ground truth** of cellular automaton behavior. It reveals the full topology of the dynamic system—its stability, cyclicity, and sensitivity to initial conditions—and serves as the empirical foundation for classifying Wolfram’s four CA classes. Modern visualization tools such as **Wuensche’s DDLab** exploit sampling, compression, and inverse algorithms to explore these structures, making the STG an indispensable framework for understanding how **local determinism gives rise to global complexity**.

In [56]:
from itertools import product

def apply_rule_to_config(config, rule_map, r=RADIUS, periodic=True):
    """
    Apply the local rule to one full configuration string.

    Parameters
    ----------
    config : str
        Current configuration, e.g. '0101'.
    rule_map : dict
        Local mapping {neighborhood: output}.
    r : int
        Neighborhood radius.
    periodic : bool
        If True, uses periodic boundary conditions (wrap-around).

    Returns
    -------
    new_config : str
        Next-step configuration Φ(config).
    """
    n = len(config)
    new_state = []
    for i in range(n):
        neighborhood = ''
        for offset in range(-r, r + 1):
            j = (i + offset) % n if periodic else min(max(i + offset, 0), n - 1)
            neighborhood += config[j]
        new_state.append(rule_map[neighborhood])
    return ''.join(new_state)


def build_state_transition_graph(rule_number=RULE, r=RADIUS, N=3, periodic=True):
    """
    Construct the State-Transition Graph for a 1D binary CA.

    Nodes  : all 2^N configurations.
    Edges  : directed edges C_t -> C_{t+1} = Φ(C_t).
    Labels : optional numeric step index (1 per step).
    """
    # ---- local rule table ----
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    rule_map = {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

    # ---- enumerate configurations ----
    configs = [''.join(p) for p in product('01', repeat=N)]
    edges = []

    for C in configs:
        nextC = apply_rule_to_config(C, rule_map, r, periodic)
        edges.append({'from': C, 'to': nextC})
    return configs, edges


# ---- Example: build STG for current global rule ----
N = 3   # configuration length (keep small: grows as 2^N)
configs, stg_edges = build_state_transition_graph(N=N)

print("============================================================")
print(f"STATE-TRANSITION GRAPH — Rule {RULE} (radius={RADIUS}, N={len(configs[0])})")
print("------------------------------------------------------------")
print("NODES (all configurations):")
for c in configs:
    print(f"  {c}")
print(f"Total nodes: {len(configs)}")

print("\nEDGES (C_t -> C_{t+1}):")
for e in stg_edges:
    print(f"  {e['from']} -> {e['to']}")
print(f"Total edges: {len(stg_edges)}")

# ---- compute attractors / basins -------------------------------------------
import networkx as nx
G = nx.DiGraph()
for e in stg_edges:
    G.add_edge(e['from'], e['to'])
cycles = list(nx.simple_cycles(G))

print("\nDetected attractors (cycles of Φ):")
for cyc in cycles:
    if len(cyc) == 1:
        print(f"  Fixed point: {cyc[0]}")
    else:
        print(f"  Cycle({len(cyc)}): {' -> '.join(cyc)}")
print(f"Total attractors: {len(cycles)}")
print("============================================================")

STATE-TRANSITION GRAPH — Rule 30 (radius=1, N=3)
------------------------------------------------------------
NODES (all configurations):
  000
  001
  010
  011
  100
  101
  110
  111
Total nodes: 8

EDGES (C_t -> C_{t+1}):
  000 -> 000
  001 -> 111
  010 -> 111
  011 -> 010
  100 -> 111
  101 -> 001
  110 -> 100
  111 -> 000
Total edges: 8

Detected attractors (cycles of Φ):
  Fixed point: 000
Total attractors: 1


## **4. Jump Graph (Perturbation Network): Mapping Transitions Among Attractors**
The **Jump Graph**, also known as a **Perturbation Network**, provides a **meta-level view** of cellular automaton dynamics by coarse-graining the State-Transition Graph (STG) into a network of **attractors or basins**. Each node represents a distinct long-term behavior—an attractor cycle or its corresponding basin of attraction—and a directed edge $A \!\to\! B$ is added if a small **perturbation** to a configuration within basin $A$ eventually evolves into basin $B$. Typical perturbations include **single-cell (bit) flips**, noise injections, or local rewrites, each modeling how robust the system’s dynamics are to disruption.

This representation transforms the static basin landscape into a **dynamic network of transitions among stable regimes**, exposing the system’s **robustness**, **sensitivity**, and **error propagation pathways**. A dense, highly connected Jump Graph suggests fragile dynamics where minor perturbations easily shift the system between attractors, while sparse or self-loop-dominated structures indicate strong stability and resistance to noise.

Practically, Jump Graphs serve as diagnostic tools for **meta-dynamics**—the “dynamics of dynamics.” They are widely used in studies of **robust computation**, **biological regulatory networks**, and **cellular automata–based cryptography**, where attractor stability is crucial. For example, in cryptographic CA, a good pseudorandom generator corresponds to a Jump Graph dominated by **a single, extremely long cycle** that attracts nearly all states, ensuring unpredictability and uniform coverage.

By summarizing how attractors transform under perturbation, the Jump Graph bridges **microscopic transitions and macroscopic resilience**, providing an elegant framework to study how **complex systems maintain or lose identity under noise**.

In [62]:
from itertools import product
import networkx as nx

def find_attractor_map(rule_number=RULE, r=RADIUS, N=3, periodic=True):
    """
    Compute attractor membership for all configurations.
    Returns:
        attractor_map: {config: attractor_id}
        attractors   : list of attractor (list of configs in cycle)
    """
    # ---- reuse STG construction ----
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    rule_map = {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

    def apply_rule(config):
        n = len(config)
        out = []
        for i in range(n):
            nb = ''.join(config[(i+o) % n] for o in range(-r, r+1))
            out.append(rule_map[nb])
        return ''.join(out)

    configs = [''.join(p) for p in product('01', repeat=N)]
    G = nx.DiGraph()
    for C in configs:
        G.add_edge(C, apply_rule(C))
    cycles = list(nx.simple_cycles(G))

    # Map each node to attractor id
    attractor_map = {}
    for idx, cyc in enumerate(cycles):
        for node in cyc:
            attractor_map[node] = idx

    # For nodes not in cycles, assign them to the basin of their eventual attractor
    for C in configs:
        if C in attractor_map:
            continue
        seen = set()
        cur = C
        while cur not in attractor_map and cur not in seen:
            seen.add(cur)
            cur = apply_rule(cur)
        attractor_map[C] = attractor_map.get(cur, -1)  # -1 if transient w/o attractor
    return attractor_map, cycles


def bit_flip(config, i):
    """Flip bit i in binary string."""
    flipped = list(config)
    flipped[i] = '1' if flipped[i] == '0' else '0'
    return ''.join(flipped)


def build_jump_graph(rule_number=RULE, r=RADIUS, N=3):
    """
    Build Jump Graph (Perturbation Network).

    Nodes: attractors (A0, A1, ...)
    Edges: A_i -> A_j if flipping one bit in any config of basin(A_i)
           eventually leads to attractor A_j (i may = j).
    """
    attractor_map, attractors = find_attractor_map(rule_number, r, N)
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    rule_map = {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

    # Build a quick transition function
    def evolve(config):
        n = len(config)
        out = []
        for i in range(n):
            nb = ''.join(config[(i+o) % n] for o in range(-r, r+1))
            out.append(rule_map[nb])
        return ''.join(out)

    # Collect edges between attractors
    edges = []
    configs = list(attractor_map.keys())
    for C in configs:
        srcA = attractor_map[C]
        for i in range(len(C)):
            perturbed = bit_flip(C, i)
            # evolve perturbed state until it reaches an attractor
            seen = set()
            cur = perturbed
            while cur not in attractor_map and cur not in seen:
                seen.add(cur)
                cur = evolve(cur)
            dstA = attractor_map.get(cur, -1)
            if srcA != -1 and dstA != -1:
                edges.append({'from': f"A{srcA}", 'to': f"A{dstA}",
                              'perturb': f"{C}→{perturbed}", 'bit': i})
    # Deduplicate
    unique_edges = {(e['from'], e['to']) for e in edges}
    edges_clean = [{'from': f, 'to': t} for f, t in sorted(unique_edges)]
    return [f"A{i}" for i in range(len(attractors))], edges_clean, attractors


# ---- Example: Build Jump Graph ----
N = 3  # number of cells
nodes, edges, attractors = build_jump_graph(N=N)

print("============================================================")
print(f"JUMP GRAPH (Perturbation Network) — Rule {RULE} (radius={RADIUS}, N={N})")
print("------------------------------------------------------------")
print("Attractor nodes:")
for i, cyc in enumerate(attractors):
    cyc_str = ' -> '.join(cyc)
    print(f"  A{i}: {cyc_str}")
print(f"Total attractors: {len(attractors)}")

print("\nEdges (A_i -> A_j):")
for e in edges:
    print(f"  {e['from']} -> {e['to']}")
print(f"Total edges: {len(edges)}")
print("============================================================")

JUMP GRAPH (Perturbation Network) — Rule 30 (radius=1, N=3)
------------------------------------------------------------
Attractor nodes:
  A0: 000
Total attractors: 1

Edges (A_i -> A_j):
  A0 -> A0
Total edges: 1


# III. Emergent Structure Representations（涌现结构图）

## **5. Subsystem / Module Graph (Particle–Reaction Network): The “Particle Physics” of Cellular Automata**
The **Subsystem or Module Graph** abstracts a cellular automaton’s complex space–time behavior into a **high-level interaction network** among its emergent structures—stable or mobile configurations such as *gliders*, *still lifes*, *oscillators*, and *guns*. In this framework, each **persistent structure** becomes a **node**, and **edges** represent possible **interactions** between them—collisions, fusions, annihilations, or transformations into other structures.

This representation captures the **effective dynamics of information carriers** within the automaton, turning microscopic rule-based evolution into a **“particle physics” graph** of discrete entities and their reaction channels. For instance, in Conway’s *Game of Life*, gliders act as mobile information packets, while their collisions can generate logic gates or memory elements. The resulting reaction network maps out all permissible interactions, forming the basis for **higher-level computation and symbolic reasoning** within the CA.

By emphasizing structure and interaction rather than raw cell updates, the subsystem graph provides a **modular, coarse-grained model** of how complex computation emerges from simple local rules. It aligns naturally with the framework of **computational mechanics**, where domains, particles, and their interactions together define the automaton’s *computational syntax*. Through this lens, the CA is viewed not just as a dynamical system, but as a **computational medium**—one whose effective laws can be read off from the topology of its particle–reaction graph.

In [68]:
from itertools import product
import networkx as nx

def evolve(config, rule_map, r=RADIUS):
    """Apply the local CA rule once under periodic boundary conditions."""
    n = len(config)
    out = []
    for i in range(n):
        nb = ''.join(config[(i+o) % n] for o in range(-r, r+1))
        out.append(rule_map[nb])
    return ''.join(out)


def find_repeating_patterns(rule_number=RULE, r=RADIUS, N=6, max_steps=30):
    """
    Detect short repeating or stable patterns ("particles") in a 1D CA.

    Parameters
    ----------
    rule_number : int
        CA rule number (e.g., 30, 90, 110)
    r : int
        Neighborhood radius
    N : int
        Number of cells in lattice (small for demo)
    max_steps : int
        Simulation depth to detect periodicity

    Returns
    -------
    particles : dict
        {particle_id: representative configuration string}
    interactions : list of dict
        Each dict: {'from': str, 'to': str, 'merge': str}
    """
    # ---- build rule map ----
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    rule_map = {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

    configs = [''.join(p) for p in product('01', repeat=N)]
    particles = {}
    cycles_seen = {}
    pid = 0

    # ---- find all periodic or stable configurations ----
    for C0 in configs:
        seen = {}
        cur = C0
        for step in range(max_steps):
            if cur in seen:
                cycle_start = seen[cur]
                cycle = list(seen.keys())[cycle_start:]
                rep = cycle[0]
                if rep not in cycles_seen:
                    cycles_seen[rep] = cycle
                    particles[f"P{pid}"] = rep
                    pid += 1
                break
            seen[cur] = step
            cur = evolve(cur, rule_map, r)

    # ---- define interactions ----
    interactions = []
    for p1_name, p1 in particles.items():
        for p2_name, p2 in particles.items():
            if p1_name == p2_name:
                continue
            # merge two patterns (simple overlap)
            merged = p1[:len(p1)//2] + p2[len(p2)//2:]
            evolved = evolve(merged, rule_map, r)
            for dst_name, dst in particles.items():
                if evolved == dst:
                    interactions.append({
                        'from': p1_name,
                        'to': dst_name,
                        'merge': f"{p1}+{p2}→{dst}"
                    })
    return particles, interactions


# ---- Example run ----
N = 4  # Lattice length
particles, interactions = find_repeating_patterns(RULE, RADIUS, N)

print("============================================================")
print(f"SUBSYSTEM / MODULE GRAPH (Particle–Reaction Graph)")
print(f"Rule {RULE} (radius={RADIUS}, N={N})")
print("------------------------------------------------------------")
print("Detected stable / repeating patterns (particles):")
for name, patt in particles.items():
    print(f"  {name}: {patt}")
print(f"Total particles: {len(particles)}")

print("\nPossible interactions (reactions):")
for e in interactions:
    print(f"  {e['from']} -> {e['to']}   ({e['merge']})")
print(f"Total reactions: {len(interactions)}")
print("============================================================")

SUBSYSTEM / MODULE GRAPH (Particle–Reaction Graph)
Rule 30 (radius=1, N=4)
------------------------------------------------------------
Detected stable / repeating patterns (particles):
  P0: 0000
  P1: 0001
  P2: 0010
  P3: 1110
  P4: 0100
  P5: 0101
  P6: 1101
  P7: 0111
  P8: 1000
  P9: 1010
  P10: 1011
Total particles: 11

Possible interactions (reactions):
  P0 -> P10   (0000+0001→1011)
  P0 -> P7   (0000+0010→0111)
  P0 -> P7   (0000+1110→0111)
  P0 -> P0   (0000+0100→0000)
  P0 -> P10   (0000+0101→1011)
  P0 -> P10   (0000+1101→1011)
  P0 -> P3   (0000+0111→1110)
  P0 -> P0   (0000+1000→0000)
  P0 -> P7   (0000+1010→0111)
  P0 -> P3   (0000+1011→1110)
  P1 -> P0   (0001+0000→0000)
  P1 -> P7   (0001+0010→0111)
  P1 -> P7   (0001+1110→0111)
  P1 -> P0   (0001+0100→0000)
  P1 -> P10   (0001+0101→1011)
  P1 -> P10   (0001+1101→1011)
  P1 -> P3   (0001+0111→1110)
  P1 -> P0   (0001+1000→0000)
  P1 -> P7   (0001+1010→0111)
  P1 -> P3   (0001+1011→1110)
  P2 -> P0   (0010+0000→0000)
 

## **6. Computational-Mechanics Graph (Domains → Particles → Interactions): Revealing the Rule’s Computational Syntax**
The **Computational Mechanics** framework, pioneered by Crutchfield and collaborators, formalizes the discovery of **hidden structure and computation** within cellular automata through a hierarchical graph representation linking **domains**, **particles**, and their **interactions**. The resulting *Computational-Mechanics Graph* depicts how information is stored, transmitted, and transformed in space–time—effectively uncovering the rule’s **computational syntax**.

The process begins with identifying **domains**, which are **regular, repeating background patterns** that define the automaton’s stable “vacuum.” These domains can be described as regular languages and represented compactly via finite-state machines. Next, by filtering out this background, researchers extract **defects or particles**—localized deviations that move through space and carry information. Finally, by observing and cataloging **particle collisions**, one constructs a **reaction graph** that maps every possible interaction and its outcome (annihilation, fusion, scattering, etc.).

Together, these layers form a *hierarchical causal grammar*:

- **Domains** define the system’s underlying order;
- **Particles** act as mobile carriers of state information;
- **Interactions** encode the system’s effective computation.

This framework has been applied extensively to rules such as **Elementary CA Rule 54**, where a full domain–particle–interaction characterization reveals logical operations, signal propagation, and even universal computation within purely local deterministic rules. The computational-mechanics graph thus provides a principled bridge from **microscopic updates** to **macroscopic information processing**, demonstrating how simple automata instantiate rich, rule-specific “languages of computation.”

In [74]:
from itertools import product
import numpy as np

def build_rule_map(rule_number=RULE, r=RADIUS):
    """Return local neighborhood → output mapping."""
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    return {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}


def evolve_once(config, rule_map, r=RADIUS):
    """Apply CA rule once under periodic boundary."""
    n = len(config)
    out = []
    for i in range(n):
        nb = ''.join(config[(i+o) % n] for o in range(-r, r+1))
        out.append(rule_map[nb])
    return ''.join(out)


def simulate_space_time(rule_number=RULE, r=RADIUS, N=20, T=20, seed="random"):
    """Generate a 2D space–time field."""
    rule_map = build_rule_map(rule_number, r)
    if seed == "random":
        row = ''.join(np.random.choice(['0','1'], N))
    else:
        row = seed
    space_time = [row]
    for _ in range(T-1):
        row = evolve_once(row, rule_map, r)
        space_time.append(row)
    return np.array([[int(c) for c in r] for r in space_time])


def detect_domains(space_time):
    """
    Very rough domain detector:
    Find spatial columns with fixed periodic patterns.
    """
    T, N = space_time.shape
    domain_cols = []
    for j in range(N):
        col = ''.join(str(space_time[t,j]) for t in range(T))
        # check for repeating subpattern (period <= 4)
        for p in range(1,5):
            if col[:p] * (len(col)//p) in col:
                domain_cols.append(j)
                break
    return domain_cols


def extract_particles(space_time, domain_cols):
    """
    Define 'particles' as local deviations from domain columns.
    """
    T, N = space_time.shape
    particles = []
    for t in range(T):
        for j in range(N):
            if j not in domain_cols:
                particles.append((t, j))
    return particles


def build_computational_mechanics_graph(rule_number=RULE, r=RADIUS, N=20, T=20):
    """
    Build a simplified Domain–Particle–Interaction graph.
    """
    space_time = simulate_space_time(rule_number, r, N, T)
    domain_cols = detect_domains(space_time)
    particles = extract_particles(space_time, domain_cols)

    # Interactions: if two particles appear adjacent in same timestep
    interactions = []
    for (t1, j1) in particles:
        for (t2, j2) in particles:
            if t1 == t2 and abs(j1 - j2) == 1:
                interactions.append({'from': f"P({t1},{j1})",
                                     'to': f"P({t2},{j2})",
                                     'type': 'collision'})
    return domain_cols, particles, interactions


# ---- Example Run ----
domain_cols, particles, interactions = build_computational_mechanics_graph(RULE, RADIUS, N=20, T=15)

print("============================================================")
print(f"COMPUTATIONAL-MECHANICS GRAPH — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
print(f"Detected domain columns (background periodic structure): {domain_cols}")
print(f"Detected {len(particles)} particle positions (local defects).")
print(f"Detected {len(interactions)} possible interactions.")
print("Sample interactions:")
for e in interactions[:10]:
    print(f"  {e['from']} -> {e['to']} ({e['type']})")
print("============================================================")

COMPUTATIONAL-MECHANICS GRAPH — Rule 30 (radius=1)
------------------------------------------------------------
Detected domain columns (background periodic structure): []
Detected 300 particle positions (local defects).
Detected 570 possible interactions.
Sample interactions:
  P(0,0) -> P(0,1) (collision)
  P(0,1) -> P(0,0) (collision)
  P(0,1) -> P(0,2) (collision)
  P(0,2) -> P(0,1) (collision)
  P(0,2) -> P(0,3) (collision)
  P(0,3) -> P(0,2) (collision)
  P(0,3) -> P(0,4) (collision)
  P(0,4) -> P(0,3) (collision)
  P(0,4) -> P(0,5) (collision)
  P(0,5) -> P(0,4) (collision)


# IV. Causal Constructions（因果结构图）

## **7. Dependency Graph (Data-Dependency Network): Mapping Computational Flow and Parallelism**
The **Dependency Graph** represents the **computational structure** underlying a cellular automaton’s update process. Each node corresponds to a **task**—specifically, the computation of a cell’s state $s_i^{t+1}$ at the next time step—and directed edges capture **data dependencies** from the input cells at time $t$ on which that computation relies. In a one-dimensional CA with radius $r$, each node $(i, t+1)$ depends on $2r{+}1$ predecessors $(i-r, t), \ldots, (i+r, t)$; the resulting directed acyclic graph (DAG) encodes the entire **dataflow pipeline** of the automaton.

This construction emphasizes **how information is propagated and computed**, rather than what pattern it produces. By examining the graph’s topology, one can immediately identify **independent subgraphs**—sets of updates that can be executed simultaneously—thereby exposing the automaton’s inherent **parallelism**. In parallel and distributed computing, such DAGs are instrumental for **task scheduling, dependency resolution, and performance optimization**.

Conceptually, the dependency graph situates cellular automata within the broader theory of **parallel dynamical systems**, where global evolution is viewed as a composition of local update tasks with explicit dependency constraints. It thus provides a bridge between **dynamical systems theory** and **parallel computation**, showing that the same local rule defining a CA’s dynamics also defines an executable **computation graph** governing dataflow, concurrency, and causality.

In [75]:
from itertools import product
import networkx as nx

def build_dependency_graph(rule_number=RULE, r=RADIUS, N=8, T=3):
    """
    Build dependency graph for a 1D CA unfolded in time.

    Parameters
    ----------
    rule_number : int
        CA rule number (e.g., 30, 110)
    r : int
        Neighborhood radius
    N : int
        Number of spatial cells
    T : int
        Number of time steps (height of unrolled computation)

    Returns
    -------
    G : networkx.DiGraph
        Dependency DAG with nodes (i, t)
    """

    G = nx.DiGraph()

    # Create nodes for all (i,t)
    for t in range(T+1):  # include initial layer
        for i in range(N):
            G.add_node((i, t))

    # Create edges: from (j,t) -> (i,t+1) if j in neighborhood of i
    for t in range(T):
        for i in range(N):
            for offset in range(-r, r+1):
                j = (i + offset) % N  # periodic boundary
                G.add_edge((j, t), (i, t+1))

    return G


# ---- Example: Build and print dependency graph ----
N = 6   # number of cells (spatial size)
T = 2   # number of time steps to unfold
G = build_dependency_graph(RULE, RADIUS, N, T)

print("============================================================")
print(f"DEPENDENCY GRAPH — Rule {RULE} (radius={RADIUS}, N={N}, T={T})")
print("------------------------------------------------------------")
print(f"Total nodes: {len(G.nodes())}")
print(f"Total edges: {len(G.edges())}\n")

print("Sample edges (data dependencies):")
for u, v in list(G.edges())[:15]:
    print(f"  {u} -> {v}")
print("============================================================")

DEPENDENCY GRAPH — Rule 30 (radius=1, N=6, T=2)
------------------------------------------------------------
Total nodes: 18
Total edges: 36

Sample edges (data dependencies):
  (0, 0) -> (0, 1)
  (0, 0) -> (1, 1)
  (0, 0) -> (5, 1)
  (1, 0) -> (0, 1)
  (1, 0) -> (1, 1)
  (1, 0) -> (2, 1)
  (2, 0) -> (1, 1)
  (2, 0) -> (2, 1)
  (2, 0) -> (3, 1)
  (3, 0) -> (2, 1)
  (3, 0) -> (3, 1)
  (3, 0) -> (4, 1)
  (4, 0) -> (3, 1)
  (4, 0) -> (4, 1)
  (4, 0) -> (5, 1)


## **8. Causal Network (Event-Level Causality Graph): Tracing the True Chains of Influence**
The **Causal Network** offers a **fine-grained, event-level representation** of a cellular automaton’s dynamics, focusing not on configurations or cells, but on **individual update events**. Each node corresponds to a specific event—“cell $i$ being updated at time $t$”—and a directed edge $A \!\to\! B$ is drawn whenever the **output of event $A$** directly contributes to the **input neighborhood** of event $B$. In this way, the network records the **actual flow of causal influence** through space–time, independent of the underlying lattice geometry or synchronous clock.

Unlike the Dependency Graph, which represents *potential* dataflow within a single time step, the Causal Network represents **realized causal relations** unfolding over many steps. It functions as a **causal history**: by tracing paths through the network, one can determine exactly which earlier events affected a given outcome, forming a causal cone analogous to those in relativistic physics. This structure is particularly valuable for **asynchronous**, **mobile**, or **non-lattice** automata, where update order and local interactions vary dynamically.

In Wolfram’s *A New Kind of Science* framework, such causal networks form the backbone of a proposed **fundamental ontology of computation and physics**, where space and time themselves are emergent from the web of event-to-event relationships. Within cellular automata research, they provide a powerful tool for analyzing **information propagation, influence, and concurrency**, revealing how deterministic local updates weave together into a global fabric of causation.

In [78]:
from itertools import product
import numpy as np
import networkx as nx

def build_rule_map(rule_number=RULE, r=RADIUS):
    """Return local neighborhood → output mapping."""
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    return {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}


def evolve_once(config, rule_map, r=RADIUS):
    """Apply CA rule once (periodic)."""
    n = len(config)
    out = []
    for i in range(n):
        nb = ''.join(config[(i+o) % n] for o in range(-r, r+1))
        out.append(rule_map[nb])
    return ''.join(out)


def build_causal_network(rule_number=RULE, r=RADIUS, N=8, T=4, seed="random"):
    """
    Build event-level causal network.
    Nodes: (i, t)
    Edges: (j, t) → (i, t+1) if flipping j changes i's next state.
    """
    # ---- generate initial configuration ----
    rule_map = build_rule_map(rule_number, r)
    if seed == "random":
        row = ''.join(np.random.choice(['0','1'], N))
    else:
        row = seed

    # ---- simulate space-time field ----
    space_time = [row]
    for _ in range(T-1):
        row = evolve_once(row, rule_map, r)
        space_time.append(row)
    space_time = np.array([[int(c) for c in r] for r in space_time])

    # ---- build causal edges ----
    G = nx.DiGraph()
    for t in range(T):
        for i in range(N):
            G.add_node((i, t))

    # For each event (i,t+1), test flipping each input cell (j,t)
    for t in range(T-1):
        current = ''.join(str(x) for x in space_time[t])
        next_state = space_time[t+1]

        for i in range(N):
            nb_indices = [(i+o) % N for o in range(-r, r+1)]
            nb = ''.join(current[j] for j in nb_indices)
            original_output = rule_map[nb]

            for j in nb_indices:
                # flip cell j at time t
                flipped = list(current)
                flipped[j] = '1' if flipped[j] == '0' else '0'
                new_nb = ''.join(flipped[k] for k in nb_indices)
                new_output = rule_map[new_nb]

                # if flipping j changes output of i, it's causal
                if new_output != original_output:
                    G.add_edge((j, t), (i, t+1))

    return G, space_time


# ---- Example run ----
N = 6
T = 3
G, space_time = build_causal_network(RULE, RADIUS, N, T)

print("============================================================")
print(f"CAUSAL NETWORK — Rule {RULE} (radius={RADIUS}, N={N}, T={T})")
print("------------------------------------------------------------")
print(f"Total events (nodes): {len(G.nodes())}")
print(f"Total causal links (edges): {len(G.edges())}\n")

print("Sample causal edges (event A → B):")
for u, v in list(G.edges())[:15]:
    print(f"  {u} -> {v}")
print("============================================================")

CAUSAL NETWORK — Rule 30 (radius=1, N=6, T=3)
------------------------------------------------------------
Total events (nodes): 18
Total causal links (edges): 28

Sample causal edges (event A → B):
  (0, 0) -> (0, 1)
  (0, 0) -> (1, 1)
  (1, 0) -> (0, 1)
  (1, 0) -> (1, 1)
  (1, 0) -> (2, 1)
  (2, 0) -> (1, 1)
  (2, 0) -> (2, 1)
  (2, 0) -> (3, 1)
  (3, 0) -> (2, 1)
  (3, 0) -> (3, 1)
  (3, 0) -> (4, 1)
  (4, 0) -> (3, 1)
  (4, 0) -> (5, 1)
  (5, 0) -> (0, 1)
  (5, 0) -> (4, 1)


## **9. ε-Machine (Minimal Predictive Architecture): The Causal States of Computation**
The **ε-Machine** is a central construct in **computational mechanics**, providing the **minimal predictive model** of a dynamical system—including cellular automata—by organizing its histories into **causal states**. Each node represents an equivalence class of *pasts* (sequences of prior observations or configurations) that yield the **same conditional distribution over futures**. Two pasts belong to the same causal state if they are indistinguishable in terms of their predictive consequences.

Edges correspond to **symbol-emitting transitions** between causal states: when a specific output symbol is produced (e.g., a cell’s next state), the system moves from one causal state to another. The resulting graph—a deterministic finite-state machine—is the **ε-Machine**, which encapsulates both the **structure and memory** of the process in the most compact possible form.

By construction, the ε-Machine achieves **maximal predictive power with minimal complexity**. Its **statistical complexity**, defined as the Shannon entropy over the causal-state distribution, quantifies the amount of information a system must store to optimally predict its future. Applied to cellular automata, ε-Machines reveal how local update rules generate long-range correlations, how ordered domains and chaotic regions coexist, and how information is stored, transmitted, and transformed across space–time.

Thus, the ε-Machine serves as the **causal architecture** underlying the automaton’s behavior: it abstracts raw configurations into an information-theoretic model of **how the system computes and remembers**, bridging local deterministic rules and emergent predictive structure.

In [79]:
from itertools import product
import numpy as np
import networkx as nx
from collections import defaultdict

def build_rule_map(rule_number=RULE, r=RADIUS):
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    return {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

def evolve_once(config, rule_map, r=RADIUS):
    n = len(config)
    out = []
    for i in range(n):
        nb = ''.join(config[(i+o) % n] for o in range(-r, r+1))
        out.append(rule_map[nb])
    return ''.join(out)

def simulate_center_series(rule_number=RULE, r=RADIUS, N=21, T=50, seed="random"):
    """Simulate CA and record center cell sequence over T steps."""
    rule_map = build_rule_map(rule_number, r)
    if seed == "random":
        row = ''.join(np.random.choice(['0','1'], N))
    else:
        row = seed
    series = []
    for _ in range(T):
        series.append(row[N//2])
        row = evolve_once(row, rule_map, r)
    return ''.join(series)

def reconstruct_epsilon_machine(series, L=3):
    """
    Reconstruct a simplified ε-machine from a binary sequence.
    L : length of past window.
    """
    transitions = defaultdict(list)
    next_symbol_probs = {}
    past_to_future = defaultdict(list)

    # 1. Collect transitions
    for t in range(len(series) - L):
        past = series[t:t+L]
        future_symbol = series[t+L]
        transitions[past].append(future_symbol)

    # 2. Estimate future distributions
    for past, futures in transitions.items():
        zeros = futures.count('0')
        ones = futures.count('1')
        p1 = ones / (zeros + ones)
        next_symbol_probs[past] = round(p1, 2)
        past_to_future[past] = (p1, 1 - p1)

    # 3. Group equivalent pasts with same future distribution
    eq_classes = {}
    state_id = 0
    for past, p1 in next_symbol_probs.items():
        if p1 not in eq_classes:
            eq_classes[p1] = f"S{state_id}"
            state_id += 1

    # 4. Build state transitions (symbol-emitting edges)
    edges = []
    for past in transitions:
        state = eq_classes[next_symbol_probs[past]]
        next_past = past[1:] + series[len(past)] if len(series) > len(past) else past[1:] + '0'
        if next_past in next_symbol_probs:
            next_state = eq_classes[next_symbol_probs[next_past]]
            symbol = transitions[past][0]
            edges.append({'from': state, 'to': next_state, 'symbol': symbol})

    # 5. Compute statistical complexity
    probs = np.ones(len(eq_classes)) / len(eq_classes)
    C_mu = -np.sum(probs * np.log2(probs))

    return eq_classes, edges, C_mu, next_symbol_probs

# ---- Example run ----
series = simulate_center_series(RULE, RADIUS, N=21, T=50)
eq_classes, edges, C_mu, dist = reconstruct_epsilon_machine(series, L=3)

print("============================================================")
print(f"ε-MACHINE — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
print(f"Center cell time series (first 40 symbols): {series[:40]}")
print("\nCausal states (equivalence classes by predictive distribution):")
for p1, s in eq_classes.items():
    print(f"  {s}: P(1|past) = {p1}")
print(f"Total causal states: {len(eq_classes)}")
print(f"Statistical complexity Cμ = {C_mu:.3f} bits")

print("\nTransitions:")
for e in edges[:10]:
    print(f"  {e['from']} -> {e['to']}  (emit '{e['symbol']}')")
print("============================================================")

ε-MACHINE — Rule 30 (radius=1)
------------------------------------------------------------
Center cell time series (first 40 symbols): 1110011101001000100101000001010011000001

Causal states (equivalence classes by predictive distribution):
  S0: P(1|past) = 0.0
  S1: P(1|past) = 0.4
  S2: P(1|past) = 0.62
  S3: P(1|past) = 0.29
  S4: P(1|past) = 0.5
  S5: P(1|past) = 0.38
  S6: P(1|past) = 0.43
Total causal states: 7
Statistical complexity Cμ = 2.807 bits

Transitions:
  S0 -> S1  (emit '0')
  S1 -> S2  (emit '0')
  S2 -> S6  (emit '1')
  S3 -> S5  (emit '1')
  S4 -> S1  (emit '1')
  S1 -> S5  (emit '0')
  S5 -> S2  (emit '0')
  S6 -> S6  (emit '1')


# V. Generalizations and Meta-Analysis（泛化与度量层）

## **10. Graph Cellular Automata (GCA): Extending CA Dynamics Beyond Regular Lattices**
The **Graph Cellular Automaton (GCA)** generalizes the classical concept of cellular automata by defining their dynamics on an **arbitrary graph** $G = (V, E)$ rather than on a regular lattice. Each node $v \in V$ acts as a **cell**, maintaining a local state, while edges define its **neighborhood**—the set of nodes whose states influence its next update. The local transition rule thus becomes a function
$$
f: \mathbb{S}^{\deg(v)} \to \mathbb{S},
$$
applied independently to every node based on the current states of its adjacent neighbors.

This formulation frees cellular automata from the constraints of spatial uniformity and periodic boundary conditions, allowing them to operate on **heterogeneous, irregular, or dynamic topologies** such as social networks, protein–interaction graphs, or evolving communication structures. GCAs preserve the spirit of CA—local interactions producing emergent global patterns—but extend it to any relational domain where the notion of “neighbor” is topological rather than geometric.

Recent advances introduce **Graph Neural Cellular Automata (GNCA)**, where the local update rule is **parameterized by a Graph Neural Network (GNN)**. In this hybrid model, message-passing layers learn how to aggregate and transform neighbor information, enabling **trainable, differentiable** versions of cellular automata. GNCA models can learn to reproduce target morphogenetic patterns, simulate physical systems, or generate self-organizing behaviors, merging **deep learning** with **complex-systems dynamics**.

In essence, GCAs transform cellular automata from a fixed-lattice formalism into a **topology-agnostic, adaptive computational framework**—one that unites discrete dynamical systems, network science, and neural representation learning under a single generalized paradigm.

In [80]:
import networkx as nx
import numpy as np
from itertools import product

def build_rule_map(rule_number=RULE, r=RADIUS):
    """Local Boolean rule map for 1D binary CA generalized to degree-based GCA."""
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    return {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

def gca_update_rule_based(G, states, rule_map, r=RADIUS):
    """
    Apply classical CA-style rule on a graph.
    Each node's new state depends on its own + r-hop neighbors.
    """
    new_states = states.copy()
    for node in G.nodes():
        neighbors = list(G.neighbors(node)) + [node]
        # keep to 2r+1 if possible
        nb_states = ''.join(str(states[n] if n < len(states) else 0) for n in neighbors[:2*r+1])
        if nb_states in rule_map:
            new_states[node] = int(rule_map[nb_states])
        else:
            # fallback: majority vote
            s = sum(int(states[n]) for n in neighbors)
            new_states[node] = 1 if s > len(neighbors)/2 else 0
    return new_states

def gca_update_neural(G, states, W_self=0.4, W_neigh=0.6):
    """
    Simple differentiable GCA (Graph Neural Cellular Automata).
    Each node aggregates neighbor mean and self signal.
    """
    new_states = states.copy().astype(float)
    for node in G.nodes():
        neigh_mean = np.mean([states[n] for n in G.neighbors(node)]) if len(list(G.neighbors(node)))>0 else states[node]
        new_states[node] = np.clip(W_self*states[node] + W_neigh*neigh_mean, 0, 1)
    return np.round(new_states)

def simulate_gca(G, steps=5, mode="rule"):
    """Simulate multiple steps of Graph Cellular Automata."""
    states = np.random.randint(0, 2, size=len(G))
    trajectory = [states.copy()]
    rule_map = build_rule_map(RULE, RADIUS)
    for _ in range(steps):
        if mode == "rule":
            states = gca_update_rule_based(G, states, rule_map)
        elif mode == "neural":
            states = gca_update_neural(G, states)
        trajectory.append(states.copy())
    return np.array(trajectory)

# ---- Example Run ----
# Build a small random graph
G = nx.erdos_renyi_graph(n=8, p=0.3, seed=1)

traj_rule = simulate_gca(G, steps=5, mode="rule")
traj_neural = simulate_gca(G, steps=5, mode="neural")

print("============================================================")
print(f"GRAPH CELLULAR AUTOMATA — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
print(f"Graph: {len(G.nodes())} nodes, {len(G.edges())} edges")
print("Initial states (step 0):", traj_rule[0])
print("Final states (rule-based):", traj_rule[-1])
print("Final states (neural-GCA):", traj_neural[-1])
print("============================================================")

GRAPH CELLULAR AUTOMATA — Rule 30 (radius=1)
------------------------------------------------------------
Graph: 8 nodes, 11 edges
Initial states (step 0): [0 0 0 1 0 0 0 0]
Final states (rule-based): [1 0 1 0 0 0 1 1]
Final states (neural-GCA): [1. 1. 0. 0. 0. 0. 0. 0.]


## **11. Color / Cell-Lattice Graph (Spatial Pattern Graph): Quantifying Structure in Space**
The **Color or Cell-Lattice Graph** treats a cellular automaton’s spatial configuration as a **graph-structured image**, where each **cell** is represented as a **node**, and edges encode **adjacency relations** (e.g., four- or eight-neighbor connectivity in 2D lattices). Each node carries its **state value** as an **attribute or color**, transforming the CA’s instantaneous configuration into a **colored graph** that can be analyzed using tools from spatial statistics, graph theory, and image analysis.

This representation shifts focus from temporal evolution to **spatial morphology**—it captures how similar states cluster, percolate, or organize across the lattice at a given time. By applying metrics such as **graph modularity, clustering coefficients, connected-component sizes, or spectral signatures**, researchers can quantitatively assess **spatial coherence**, **domain boundaries**, and **pattern anisotropy**. For multi-state or continuous automata, color encodes richer attributes (e.g., gradients or physical quantities), extending the approach to chemical, ecological, or physical simulations.

The Color/Cell-Lattice Graph thus provides a **bridge between cellular automata and spatial data analysis**: it frames the CA configuration as a networked pattern amenable to **community detection, motif analysis, or geometric quantification**. This viewpoint is particularly valuable for 2D and higher-dimensional systems—such as *Game of Life* or morphogenetic GCAs—where the goal is to measure not just temporal dynamics but the **geometry of emergent spatial organization**.

In [88]:
import networkx as nx
import numpy as np
from itertools import product

def build_rule_map(rule_number=RULE, r=RADIUS):
    """Return local neighborhood → output mapping."""
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    return {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

def evolve_2d_step(state, rule_map, r=RADIUS):
    """
    Apply 1D CA rule row-wise on a 2D grid to create spatial pattern layers.
    (This produces a pseudo-2D CA pattern based on 1D rule replication.)
    """
    rows, cols = state.shape
    new_state = state.copy()
    for i in range(rows):
        for j in range(cols):
            nb = ''.join(str(state[i, (j+o) % cols]) for o in range(-r, r+1))
            new_state[i, j] = int(rule_map[nb])
    return new_state

def build_color_graph(rule_number=RULE, r=RADIUS, size=(10, 10), steps=5):
    """
    Build a color (cell-lattice) graph for a 2D pattern evolution.
    Each cell = node; color attribute = final state value.
    """
    rule_map = build_rule_map(rule_number, r)
    # initialize random binary grid
    state = np.random.randint(0, 2, size)
    for _ in range(steps):
        state = evolve_2d_step(state, rule_map, r)

    rows, cols = state.shape
    G = nx.grid_2d_graph(rows, cols)
    for (i, j) in G.nodes():
        G.nodes[(i, j)]["color"] = int(state[i, j])
    return G, state

G, state = build_color_graph(RULE, RADIUS, size=(6, 6), steps=5)

print("============================================================")
print(f"COLOR / CELL-LATTICE GRAPH — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
print(f"Grid size: {state.shape[0]} × {state.shape[1]}")
print("Sample node colors (state values):")
for node, data in list(G.nodes(data=True))[:10]:
    print(f"  Node {node}: color={data['color']}")
print(f"Total nodes: {len(G.nodes())}, edges: {len(G.edges())}")
print("============================================================")

COLOR / CELL-LATTICE GRAPH — Rule 30 (radius=1)
------------------------------------------------------------
Grid size: 6 × 6
Sample node colors (state values):
  Node (0, 0): color=0
  Node (0, 1): color=1
  Node (0, 2): color=0
  Node (0, 3): color=0
  Node (0, 4): color=1
  Node (0, 5): color=0
  Node (1, 0): color=0
  Node (1, 1): color=1
  Node (1, 2): color=0
  Node (1, 3): color=1
Total nodes: 36, edges: 60


## **12. Network-Science Meta-Analysis: Quantifying Complexity Across CA-Derived Graphs**
The **Network-Science Meta-Analysis** layer extends graph-based representations of cellular automata into a **quantitative analytical framework**. Once a CA is mapped into any graph form—be it a **De Bruijn graph**, **state-transition graph**, **causal network**, or **particle-interaction network**—one can apply standard **network-science metrics** to characterize and compare the system’s structural complexity.

Key measures include:

- **Degree distributions and centralities** (in-degree, betweenness, eigenvector): reveal influential configurations or causal bottlenecks.
- **Community structure and modularity:** identify clusters of tightly coupled states or coherent dynamical regimes.
- **Motif analysis:** detects overrepresented subgraph patterns, the fundamental “building blocks” of rule syntax or causal logic.
- **Spectral and entropy-based measures:** quantify hierarchy, redundancy, and overall organization within the graph.

This meta-analytic approach enables **cross-representation and cross-rule comparison**—different CAs or different graph types can be benchmarked using a shared vocabulary of network descriptors. For example, highly modular or motif-rich causal networks correspond to **structured, low-entropy dynamics**, while dense, uncorrelated connectivity indicates **chaotic or random behavior**.

In essence, network-science meta-analysis provides the final link in the chain
$$
\text{Dynamics} \;\rightarrow\; \text{Graph Representation} \;\rightarrow\; \text{Network Metrics} \;\rightarrow\; \text{Behavioral Complexity},
$$
allowing researchers to move from qualitative pattern observation to **quantitative, reproducible complexity assessment** across the entire landscape of cellular automata.

In [89]:
import networkx as nx
import numpy as np

def analyze_network_metrics(G):
    """
    Compute key network-science metrics for a given graph G.
    Supports both directed and undirected graphs.
    Returns a summary dictionary of results.
    """
    is_directed = G.is_directed()
    summary = {}

    # ---- Basic structure ----
    summary["nodes"] = G.number_of_nodes()
    summary["edges"] = G.number_of_edges()
    summary["density"] = nx.density(G)

    # ---- Degree statistics ----
    degrees = dict(G.degree())
    summary["avg_degree"] = np.mean(list(degrees.values()))

    # ---- Centralities ----
    if is_directed:
        summary["in_degree_centrality"] = np.mean(list(nx.in_degree_centrality(G).values()))
        summary["out_degree_centrality"] = np.mean(list(nx.out_degree_centrality(G).values()))
    else:
        summary["degree_centrality"] = np.mean(list(nx.degree_centrality(G).values()))

    # ---- Betweenness / Closeness ----
    summary["avg_betweenness"] = np.mean(list(nx.betweenness_centrality(G).values()))
    summary["avg_closeness"] = np.mean(list(nx.closeness_centrality(G).values()))

    # ---- Clustering and components ----
    try:
        summary["avg_clustering"] = nx.average_clustering(G.to_undirected())
    except Exception:
        summary["avg_clustering"] = None

    try:
        summary["num_components"] = nx.number_connected_components(G.to_undirected())
    except Exception:
        summary["num_components"] = nx.number_weakly_connected_components(G)

    # ---- Path length / diameter ----
    try:
        largest_cc = max(nx.connected_components(G.to_undirected()), key=len)
        subG = G.subgraph(largest_cc)
        summary["avg_path_length"] = nx.average_shortest_path_length(subG)
        summary["diameter"] = nx.diameter(subG)
    except Exception:
        summary["avg_path_length"] = None
        summary["diameter"] = None

    # ---- Community structure ----
    try:
        import community as community_louvain
        partition = community_louvain.best_partition(G.to_undirected())
        num_comm = len(set(partition.values()))
        summary["num_communities"] = num_comm
    except Exception:
        summary["num_communities"] = None

    # ---- Motif-like local structure ----
    tri = sum(nx.triangles(G.to_undirected()).values()) / 3
    summary["num_triangles"] = int(tri)

    return summary

# ---- Example: apply to a CA-derived graph (e.g., dBG) ----
from itertools import product

def debruijn_graph(rule_number=RULE, r=RADIUS):
    """Simple De Bruijn Graph builder for demonstration."""
    rule_bits = f"{rule_number:0{2**(2*r+1)}b}"[::-1]
    neighborhoods = [''.join(p) for p in product('01', repeat=2*r+1)]
    rule_map = {nb: rule_bits[i] for i, nb in enumerate(neighborhoods)}

    G = nx.DiGraph()
    nodes = [''.join(p) for p in product('01', repeat=2*r)]
    for nb, out in rule_map.items():
        u, v = nb[:-1], nb[1:]
        G.add_edge(u, v, label=out)
    return G

# Build a test graph (De Bruijn Graph for Rule 30)
G_dbg = debruijn_graph(RULE, RADIUS)
metrics = analyze_network_metrics(G_dbg)

print("============================================================")
print(f"NETWORK-SCIENCE META-ANALYSIS — Rule {RULE} (radius={RADIUS})")
print("------------------------------------------------------------")
for k, v in metrics.items():
    print(f"{k:20s}: {v}")
print("============================================================")

NETWORK-SCIENCE META-ANALYSIS — Rule 30 (radius=1)
------------------------------------------------------------
nodes               : 4
edges               : 8
density             : 0.6666666666666666
avg_degree          : 4.0
in_degree_centrality: 0.6666666666666666
out_degree_centrality: 0.6666666666666666
avg_betweenness     : 0.25
avg_closeness       : 0.675
avg_clustering      : 0.8333333333333333
num_components      : 1
avg_path_length     : 1.5
diameter            : 2
num_communities     : None
num_triangles       : 2
